# Climate-Friendly Food Systems (CFFS) Labelling Project

### The University of British Columbia

#### Created by Silvia Huang, CFFS Data Analyst
****

## Part II: Data Cleaning

## Set up and Import Libraries

In [1]:
#pip install -r requirements.txt

In [2]:
import numpy as np
import pandas as pd
import pdpipe as pdp
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest
from datetime import datetime

C:\Users\smvan\anaconda3\lib\site-packages\pdpipe\col_generation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
# Set the root path, change the the current working directory into the project folder
path = "C:/Users/smvan/CFFS-S23/CFFS-22-23"
# path = os.getcwd()
print(path)
os.chdir(path)

C:/Users/smvan/CFFS-S23/CFFS-22-23


In [4]:
# Enable reading data table in the scrolling window if you prefer
pd.set_option("display.max_rows", None, "display.max_columns", None)

***

## Import Preprocessed Datasets

In [5]:
# Read Items_List.csv
# Reading csv file: C:/Users/smvan/CFFS-S23/CFFS-22-23/data/preprocessed/Items_List.csv
Items = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Items_List.csv"))
Items.dtypes

ItemId             object
Description        object
CaseQty           float64
CaseUOM            object
PakQty            float64
PakUOM             object
InventoryGroup     object
dtype: object

In [6]:
# Display first 5 rows
Items.head()

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-2110,CHEESE CHED MED SHRED COL,2.0,bag,2.5,Kg,DAIRY
1,I-4027,CHICK THIGH B/LS/L HALAL,1.0,Kg,1.0,Kg,POULTRY
2,I-3237,CRACKER SODA SALTED IND TFC,500.0,pak,1.0,pak,FOOD - GROCERY
3,I-3388,OIL CANOLA ROMANO TFC,6.0,can,3.0,L,FOOD - GROCERY
4,I-3642,PEPPER BLK COARSE GRD SHK,12.0,ea,500.0,g,SPICES


In [7]:
# There are 486 rows and 7 columns
Items.shape

(913, 7)

In [8]:
# Read Ingredients_List.csv
Ingredients = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Ingredients_List.csv"))
Ingredients.dtypes

IngredientId     object
Qty             float64
Uom              object
Conversion      float64
InvFactor       float64
Recipe           object
dtype: object

In [9]:
# Display first 5 rows
Ingredients.head()

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,I-3388,150.0,ml,0.001,75.0,P-41914
1,I-3642,5.0,g,1.000,2.5,P-41914
2,I-4027,2.0,Kg,1.000,1.0,P-41914
3,I-6026,20.0,g,1.000,10.0,P-41914
4,I-3237,1.0,pak,1.000,1.0,R-22896


In [10]:
# There are 3278 rows and 6 columns
Ingredients.shape

(3760, 6)

In [11]:
# Read Preps_List.csv
Preps = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Preps_List.csv"))
Preps.dtypes

PrepId             object
Description        object
PakQty            float64
PakUOM             object
InventoryGroup     object
dtype: object

In [12]:
# Display first 5 rows
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-41914,GRL|Grilled Chicken Thigh,2.000,Kg,FEAST
1,P-56387,BATCH| Szechuan Green Beans,500.000,g,NaN
2,P-26682,BATCH|Black/White Sesame Seeds,907.186,g,PREP
3,P-44438,BATCH|Garlic Bread|Baguette,1.000,PTN,NaN
4,P-47958,BATCH|Greek Salad|BASE,10.000,Kg,NaN


In [13]:
# There are 546 rows and 5 columns
Preps.shape

(581, 5)

In [14]:
# Read Product_List.csv that was created from 1_data preprocessing
Products = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Products_List.csv"))
Products.dtypes

ProdId         object
Description    object
SalesGroup     object
dtype: object

In [15]:
# Display first 5 rows
Products.head()

,ProdId,Description,SalesGroup
0,R-55408,ADD|Cheese,ADD ONS
1,R-22896,ADD|Crackers,ADD ONS
2,R-53006,HS|ADD|Chicken Thigh (2),ADD ONS
3,R-52751,BAKE|Bagel Asst,FT BAKERY
4,R-19583,BAKE|Cake|Ponderosa,FT BAKERY


In [16]:
# There are 223 rows and 3 columns
Products.shape

(501, 3)

In [17]:
# Read Coversions_List.csv that was created from 1_data preprocessing
Conversions = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Conversions_List.csv"))
Conversions.dtypes

ConversionId       object
Multiplier        float64
ConvertFromQty    float64
ConvertFromUom     object
ConvertToQty      float64
ConvertToUom       object
dtype: object

In [18]:
# Display first 5 rows
Conversions.head()

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,NaN,1.000000,1.0,XXX,1.00,L
1,NaN,0.877193,1.0,1.14L,1.14,L
2,NaN,0.666667,1.0,1.5L,1.50,L
3,NaN,0.571429,1.0,1.75 L,1.75,L
4,NaN,0.500000,1.0,2L,2.00,L


In [19]:
# There are 270 rows and 6 columns
Conversions.shape

(339, 6)

***
## Update Conversion List

In [20]:
# Add the specific conversion info from the newly-processed data to a unit conversion database
Update_Conv = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "update", "Conv_UpdateConv.csv"))
Update_Conv

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,I-1028,0.008333,1.000,CT,120.0000,g
1,I-1034,0.008333,1.000,CT,120.0000,g
2,I-1035,0.010000,1.000,CT,100.0000,g
3,I-10605,0.008850,1.000,CT,113.0000,g
4,I-1126,0.006667,1.000,CT,150.0000,g
5,I-1127,0.006667,1.000,CT,150.0000,g
6,I-1141,0.013333,1.000,CT,75.0000,g
7,I-1143,0.013333,1.000,CT,75.0000,g
8,I-11519,0.020000,1.000,bag,50.0000,g
9,I-1152,0.013333,1.000,CT,75.0000,g


In [21]:
# return dataframe with null values
# There are no null values
subset_conv = Update_Conv[Update_Conv["Multiplier"].isna()]
subset_conv

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [22]:
# takes a data frame and assigns a new column called Multiplier to the dataframe
# uses iterrows() to iterate through the rows.Then subset_conv will have a new column named Multiplier with the computed values.
def assign_multiplier(df):
    for ind, row in df.iterrows():
        df.loc[ind, "Multiplier"] = row["ConvertFromQty"] / row["ConvertToQty"]
        
assign_multiplier(subset_conv)

# May 8th change: We want to 
assign_multiplier(Update_Conv)
#subset_conv

In [23]:
# Display the first 339 rows and all the columns
Update_Conv = Update_Conv.iloc[0:339, :]
Update_Conv

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,I-1028,0.008333,1.000,CT,120.0000,g
1,I-1034,0.008333,1.000,CT,120.0000,g
2,I-1035,0.010000,1.000,CT,100.0000,g
3,I-10605,0.008850,1.000,CT,113.0000,g
4,I-1126,0.006667,1.000,CT,150.0000,g
5,I-1127,0.006667,1.000,CT,150.0000,g
6,I-1141,0.013333,1.000,CT,75.0000,g
7,I-1143,0.013333,1.000,CT,75.0000,g
8,I-11519,0.020000,1.000,bag,50.0000,g
9,I-1152,0.013333,1.000,CT,75.0000,g


In [24]:
# convert dataframe to csv file
Update_Conv = pd.concat([Update_Conv, subset_conv], axis=0)
Update_Conv.to_csv("data/cleaning/update/Conv_UpdateConv.csv", index=False)

In [25]:
# retrieves the value in the 'ConversionId' column using Update_Conv.loc[index, 'ConversionId'] 
# and assigns it to the variable Id. 

# row: a Series (1-d array in pandas) You can access the individual values of the row using the column headers as keys to the
# like row['column_name'].

# if the IDs of Update_Conv are the same IDs as Conversions then we drop it from the Conversions dataframe

for index, row in Update_Conv.iterrows():
    Id = Update_Conv.loc[index, 'ConversionId']
    Conversions.drop(Conversions[Conversions['ConversionId'] == Id].index, inplace = True)

In [26]:
# combine two data frames, reset the index and remove any duplicates in the concatenated data frame, 
# and return the resulting data frame.

# Added comments below on May 8th

# frames variable is created as a list of two DataFrames, Conversions and Update_Conv.
# pd.concat() function is used to concatenate the two DataFrames vertically (i.e., stack one on top of the other). 

# drop_duplicates() method is used to remove any duplicate rows from the concatenated DataFrame based on all columns. 
# The resulting DataFrame has only unique rows.

# The resulting DataFrame is assigned back to the variable Conversions,

# drop = True ensures that the old index is not added as a new column to the dataframe 
# inplace=False parameter ensures that a new DataFrame is returned rather than modifying 
# the original Conversions DataFrame in place.

frames = [Conversions, Update_Conv]
Conversions = pd.concat(frames).reset_index(drop=True, inplace=False).drop_duplicates()

In [27]:
Conversions

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,NaN,1.000000,1.0000,XXX,1.0000,L
1,NaN,0.877193,1.0000,1.14L,1.1400,L
2,NaN,0.666667,1.0000,1.5L,1.5000,L
3,NaN,0.571429,1.0000,1.75 L,1.7500,L
4,NaN,0.500000,1.0000,2L,2.0000,L
5,NaN,0.250000,1.0000,4L,4.0000,L
6,NaN,0.083333,1.0000,FOOT,12.0000,INCH
7,NaN,0.062500,1.0000,16L,16.0000,L
8,NaN,0.059172,1.0000,1/2LTR,16.9000,fl oz
9,NaN,0.039370,1.0000,750ML,25.4000,fl oz


In [28]:
# Convert the DataFrame: Conversions into a csv file called Conversions_Added.csv
path = os.path.join(os.getcwd(), "data", "cleaning", "Conversions_Added.csv")
Conversions.to_csv(path, index = False, header = True)

### Create Unit Converter

In [29]:
# Import standard unit conversion information and construct a dataframe
Std_Unit = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "standard_conversions.csv"))
Std_Unit.head()

,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,4.92890,1,tsp,4.92890,ml
1,14.78700,1,Tbsp,14.78700,ml
2,946.35000,1,qt,946.35000,ml
3,473.17625,1,pt,473.17625,ml
4,28.34950,1,oz,28.34950,g


In [30]:
# Seperate uoms that converted to 'ml' or 'g'
# Below we create 2 lists. 
# list_unit contains list of unit of measurements that are being converted to milliliters 
# solid_unit contains a list of unit of measurements that are being converted to grams
# tolist() converts a Pandas Series or an array to a python list. 

liquid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'ml', 'ConvertFromUom'].tolist()
solid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'g', 'ConvertFromUom'].tolist()

In [31]:
# Construct a standard unit converter
def std_converter(qty, uom):
    if uom in Std_Unit['ConvertFromUom'].tolist():
        multiplier = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'Multiplier']
        Qty = float(qty)*float(multiplier)
        Uom = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'ConvertToUom'].values[0]
    else:
        Qty = qty
        Uom = uom
    return (Qty, Uom)

In [32]:
# Convert pounds to grams
std_converter(0.25,'lb')

(113.398, 'g')

In [33]:
# Test the std_converter
assert std_converter(0.25,'lb') == (113.398, 'g')

In [34]:
# Construct a unit converter for specific ingredients

# After this line below, spc_cov contains only the non-empty values from the 'ConversionId' column of the Conversions DataFrame.
spc_cov = list(filter(None, Conversions['ConversionId'].tolist()))


# Comments for spc_converter:
# The function checks if ingredient is in the liquid_unit or solid_unit lists. If so, it calls std_converter(qty, uom) to 
# convert the quantity and UOM to a standardized unit.

# If uom is not in liquid_unit or solid_unit it checks if ingre is in spc_cov, if it is and the ConvertToUom is equal to grams 
# then the function applies the factor to the qty argument to convert it to the standardized unit, and returns the result as
# a tuple containing the converted quantity and uom. If no conversion found, then it calls std_converter(qty, uom)

# If uom not in liquid_unit or solid_unit and if ingre is not in spc_cov then the function calls std_converter(qty, uom)

def spc_converter(ingre, qty, uom):
    if uom in liquid_unit + solid_unit:
        return std_converter(qty, uom)
    elif ingre in spc_cov:
        conversion = Conversions.loc[(Conversions['ConversionId'] == ingre) & (Conversions['ConvertFromUom'] == uom)
                                    & (Conversions['ConvertToUom'] == 'g')]
        multiplier = conversion['Multiplier']
        if multiplier.empty:
            return std_converter(qty, uom)
        else: 
            Qty = float(qty)/float(multiplier)
            Uom = conversion['ConvertToUom'].values[0]
            return (Qty, Uom)
    else:
        return std_converter(qty, uom)

In [35]:
spc_cov

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 'I-3237',
 'I-3388',
 'I-3388',
 'I-3388',
 'I-3642',
 'I-1197',
 'I-1813',
 'I-1813',
 'I-1820',
 'I-1820',
 'I-1821',
 'I-1821',
 'I-2220',
 'I-2220',
 'I-2220',
 'I-2254',
 'I-2254',
 'I-2933',
 'I-3223',
 'I-3223',
 'I-3224',
 'I-3224',
 'I-3225',
 'I-3225',
 'I-3229',
 'I-3283',
 'I-3283',
 'I-3348',
 'I-3370',
 'I-3386',
 'I-3386',
 'I-3386',
 'I-3389',
 'I-3389',
 'I-3389',
 'I-3392',
 'I-3392',
 'I-3392',
 'I-3458',
 'I-3501',
 'I-3545',
 'I-3545',
 'I-3563',
 'I-3572',
 'I-3572',
 'I-3589',
 'I-3589',
 'I-3590',
 'I-3591',
 'I-3591',
 'I-3617',
 'I-3617',
 'I-3619',
 'I-3620',
 'I-3621',
 'I-3624',
 'I-3624',
 'I-3625',
 'I-3625',
 'I-3628',
 'I-3628',
 'I-3629',
 'I-3629',
 'I-3630',
 'I-3632',
 'I-3632',
 'I-3638',
 'I-3638',
 'I-3643',
 'I-3644',
 'I-3646

In [36]:
# spc_cov2 is the same as spc_cov but without null values labelled as "nan" in the list
import math
spc_cov2 = [item for item in spc_cov if not(pd.isnull(item)) == True]
spc_cov2

['I-3237',
 'I-3388',
 'I-3388',
 'I-3388',
 'I-3642',
 'I-1197',
 'I-1813',
 'I-1813',
 'I-1820',
 'I-1820',
 'I-1821',
 'I-1821',
 'I-2220',
 'I-2220',
 'I-2220',
 'I-2254',
 'I-2254',
 'I-2933',
 'I-3223',
 'I-3223',
 'I-3224',
 'I-3224',
 'I-3225',
 'I-3225',
 'I-3229',
 'I-3283',
 'I-3283',
 'I-3348',
 'I-3370',
 'I-3386',
 'I-3386',
 'I-3386',
 'I-3389',
 'I-3389',
 'I-3389',
 'I-3392',
 'I-3392',
 'I-3392',
 'I-3458',
 'I-3501',
 'I-3545',
 'I-3545',
 'I-3563',
 'I-3572',
 'I-3572',
 'I-3589',
 'I-3589',
 'I-3590',
 'I-3591',
 'I-3591',
 'I-3617',
 'I-3617',
 'I-3619',
 'I-3620',
 'I-3621',
 'I-3624',
 'I-3624',
 'I-3625',
 'I-3625',
 'I-3628',
 'I-3628',
 'I-3629',
 'I-3629',
 'I-3630',
 'I-3632',
 'I-3632',
 'I-3638',
 'I-3638',
 'I-3643',
 'I-3644',
 'I-3646',
 'I-3654',
 'I-3654',
 'I-3659',
 'I-3659',
 'I-3660',
 'I-3660',
 'I-3671',
 'I-3671',
 'I-3679',
 'I-3679',
 'I-3679',
 'I-3679',
 'I-3681',
 'I-3695',
 'I-3695',
 'I-3695',
 'I-3698',
 'I-3698',
 'I-3698',
 'I-3698',

In [37]:
# Here we pass in the ingredient (I-1120) the quantity of the ingredient (1) and the unit of measurement: CT -> count
# This gives us the number of grams of that ingredient
spc_converter('I-1120', 1, 'CT')

(50.0, 'g')

In [38]:
# Identify the ones that are not in the conversion list 

In [39]:
Conversions.loc[Conversions["ConversionId"] == "I-14190"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
641,I-14190,0.001667,1.0,LOAF,600.0,g


In [40]:
c_list = Conversions["ConversionId"].unique()
"I-68700" in c_list

True

In [41]:
spc_converter("I-14190", 1, "LOAF")

(600.0, 'g')

In [42]:
# Test the spc_converter
# spc_converter('I-1120', 1, 'CT')
#assert spc_converter('I-1120', 1, 'CT') == (134.99995275001655, 'g')

***
## Items with Non-standard Units

In [43]:
# Filter out the items whose unit information is unknown 

# We find the column names
col_names = list(Ingredients.columns.values)

# Create a Items_Nonstd list
Items_Nonstd = []

# If the unit of measurement is not grams or ml and ingredient id starts with I and the ingredient is not in ConversionId column of Conversions 
# then we add it to Items_Nonstd list
for index, row in Ingredients.iterrows():
    Ingre = Ingredients.loc[index,'IngredientId']
    Uom = Ingredients.loc[index,'Uom']
    if Uom not in ['g', 'ml'] and Uom not in liquid_unit + solid_unit and Ingre.startswith('I') and Ingre not in Conversions["ConversionId"].tolist():
        Dict = {}
        Dict.update(dict(row))
        Items_Nonstd.append(Dict)

# Create a DataFrame from Items_Nonstd list
Items_Nonstd = pd.DataFrame(Items_Nonstd, columns = col_names)
# Remove duplicate ingredients of the same properties so that Items_Nonstd has only unique rows. 
Items_Nonstd.drop_duplicates(subset=['IngredientId'], inplace=True,)
Items_Nonstd

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,I-1050,1.000,CT,1.0,1.0000,P-22031
1,I-1062,0.033,SHEET,1.0,0.0330,P-22034
2,I-1059,0.042,SHEET,1.0,0.0420,P-22045
3,I-1060,0.033,CAKE,1.0,0.0330,P-22046
4,I-1061,0.042,SHEET,1.0,0.0420,P-22047
5,I-2668,0.167,PIE,1.0,0.1670,P-22058
6,I-2671,0.167,PIE,1.0,0.1670,P-22060
7,I-19633,1.000,pak,1.0,0.1818,P-23837
8,I-22489,1.000,ea,1.0,1.0000,P-32073
9,I-2598,1.000,CT,1.0,1.0000,P-38314


In [44]:
# Convert the Items_Nonstd DataFrame to a csv file
path = os.path.join(os.getcwd(), "data", "cleaning", "Items_Nonstd.csv")
Items_Nonstd.to_csv(path, index = False, header = True)

***
## Clean Preps Units

In [45]:
Preps

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-41914,GRL|Grilled Chicken Thigh,2.000,Kg,FEAST
1,P-56387,BATCH| Szechuan Green Beans,500.000,g,NaN
2,P-26682,BATCH|Black/White Sesame Seeds,907.186,g,PREP
3,P-44438,BATCH|Garlic Bread|Baguette,1.000,PTN,NaN
4,P-47958,BATCH|Greek Salad|BASE,10.000,Kg,NaN
5,P-56398,BATCH|Guacamole,2.500,Kg,PREP
6,P-54677,BATCH|Marinated Tofu|GRILLED,6.800,Kg,NaN
7,P-56766,BATCH|oven roasted tomatoes|MC,1.000,lb,NaN
8,P-42354,BATCH|Pulled Pork Braise,36.000,Kg,PREP
9,P-47654,BATCH|Vegetarian Furikake,2.310,Kg,NaN


In [46]:
pd.read_csv("data/cleaning/update/Preps_UpdateUom.csv")

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54697,LEMON|Wedge 1/8,8.00,each,PREP,84.000,g
1,P-35132,MARINATED|Lemon & Herb Chx,185.00,ea,PREP,24050.000,g
2,P-51992,YIELD|Bread|Sourdough 5/8,36.00,slice,NaN,1620.000,g
3,P-26234,BATCH|Roasted Garlic Bread,16.00,ea,PREP,1280.000,g
4,P-26170,GRILLED|NaanBread,1.00,ea,PREP,125.000,g
5,P-16305,YIELD|Smokie (1pc),1.00,ea,NaN,112.000,g
6,P-26047,BOILED|Hard Boiled Eggs FT,50.00,ea,PREP,2500.000,g
7,P-26631,BRK|Pancake|Chocolate Chip,24.00,ea,NaN,4080.000,g
8,P-26057,FRIED|Sunny/Overeasy Egg,1.00,ea,ZDONT USE OK - PREP,50.000,g
9,P-26056,GRILLED|Tomato,1.00,ea,ZDONT USE OK - PREP,90.000,g


In [47]:
# Creates 2 new columns called StdQty and StdUom in the Preps DataFrame. These columns contain NaN values
# Preparing to fill in these columns with standardized quantities and units of measurement 
Preps['StdQty'] = np.nan
Preps['StdUom'] = np.nan

In [48]:
# Convert uom into 'g' or 'ml' for each prep using the unit converter

# Retrieve the PrepId, PakQty, and PakUOM from the current row
# Pass these values to spc_converter, then we update the StdQty and StdUom columns of the current row with the converted values.
for index in Preps.index:
    PrepId = Preps.loc[index,'PrepId']
    Qty = Preps.loc[index,'PakQty']
    Uom = Preps.loc[index,'PakUOM']
    Preps.loc[index,'StdQty'] = spc_converter(PrepId, Qty, Uom)[0]
    Preps.loc[index,'StdUom'] = spc_converter(PrepId, Qty, Uom)[1]

In [49]:
Preps

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-41914,GRL|Grilled Chicken Thigh,2.000,Kg,FEAST,2000.00000,g
1,P-56387,BATCH| Szechuan Green Beans,500.000,g,NaN,500.00000,g
2,P-26682,BATCH|Black/White Sesame Seeds,907.186,g,PREP,907.18600,g
3,P-44438,BATCH|Garlic Bread|Baguette,1.000,PTN,NaN,500.00000,g
4,P-47958,BATCH|Greek Salad|BASE,10.000,Kg,NaN,10000.00000,g
5,P-56398,BATCH|Guacamole,2.500,Kg,PREP,2500.00000,g
6,P-54677,BATCH|Marinated Tofu|GRILLED,6.800,Kg,NaN,6800.00000,g
7,P-56766,BATCH|oven roasted tomatoes|MC,1.000,lb,NaN,453.59200,g
8,P-42354,BATCH|Pulled Pork Braise,36.000,Kg,PREP,36000.00000,g
9,P-47654,BATCH|Vegetarian Furikake,2.310,Kg,NaN,2310.00000,g


In [50]:
# Save cleaned preps list to file
path = os.path.join(os.getcwd(), "data", "cleaning", "Preps_Unit_Cleaned.csv")
Preps.to_csv(path, index = False, header = True)

In [51]:
pd.read_csv("data/cleaning/update/Preps_UpdateUom.csv")

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54697,LEMON|Wedge 1/8,8.00,each,PREP,84.000,g
1,P-35132,MARINATED|Lemon & Herb Chx,185.00,ea,PREP,24050.000,g
2,P-51992,YIELD|Bread|Sourdough 5/8,36.00,slice,NaN,1620.000,g
3,P-26234,BATCH|Roasted Garlic Bread,16.00,ea,PREP,1280.000,g
4,P-26170,GRILLED|NaanBread,1.00,ea,PREP,125.000,g
5,P-16305,YIELD|Smokie (1pc),1.00,ea,NaN,112.000,g
6,P-26047,BOILED|Hard Boiled Eggs FT,50.00,ea,PREP,2500.000,g
7,P-26631,BRK|Pancake|Chocolate Chip,24.00,ea,NaN,4080.000,g
8,P-26057,FRIED|Sunny/Overeasy Egg,1.00,ea,ZDONT USE OK - PREP,50.000,g
9,P-26056,GRILLED|Tomato,1.00,ea,ZDONT USE OK - PREP,90.000,g


### Get Preps with Nonstandard Unit

In [52]:
col_names = list(Preps.columns.values)
Preps_Nonstd = []

for index, row in Preps.iterrows():
    StdUom = Preps.loc[index,'StdUom']
    if StdUom not in ['g', 'ml']:
        Dict = {}
        Dict.update(dict(row))
        Preps_Nonstd.append(Dict)

Preps_Nonstd = pd.DataFrame(Preps_Nonstd, columns = col_names)

In [53]:
Preps_Nonstd

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54707,POP2|Chocolate Tart,40.0,piece,NaN,40.0,piece
1,P-22035,TP - DESSERT BAR APPLE CRUMBLE,1.0,slice,BAKING-RAW INGREDIENTS,1.0,slice
2,P-22028,TP - DESSERT BAR BUTTER TART,1.0,slice,BAKING-RAW INGREDIENTS,1.0,slice
3,P-22045,TP - DESSERT BAR LEMON CRUNCH,1.0,slice,BAKING-RAW INGREDIENTS,1.0,slice
4,P-22047,TP - DESSERT BAR MIXED BERRY,1.0,slice,BAKING-RAW INGREDIENTS,1.0,slice
5,P-32073,TP - DESSERT BUTTER TART,1.0,ea,BAKING-RAW INGREDIENTS,1.0,ea
6,P-22036,TP - DESSERT CAKE BLACK VELVET,1.0,slice,BAKING-RAW INGREDIENTS,1.0,slice
7,P-22031,TP - DESSERT CAKE BROWNIE BOMB,1.0,slice,BAKING-RAW INGREDIENTS,1.0,slice
8,P-22043,TP - DESSERT CAKE CHEESECAKE,1.0,slice,BAKING-RAW INGREDIENTS,1.0,slice
9,P-22046,TP - DESSERT CAKE MAPLE CHOCO,1.0,slice,BAKING-RAW INGREDIENTS,1.0,slice


In [54]:
# Filter out preps with nonstandard uom but have information already
Manual_PrepU = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "update", "Preps_UpdateUom.csv"))

col_names = list(Preps_Nonstd.columns.values)
Preps_Nonstd_na = []

for index, row in Preps_Nonstd.iterrows():
    PrepId = Preps_Nonstd.loc[index,'PrepId']
    if PrepId not in Manual_PrepU['PrepId'].values:
        Dict = {}
        Dict.update(dict(row))
        Preps_Nonstd_na.append(Dict)

Preps_Nonstd = pd.DataFrame(Preps_Nonstd_na, columns = col_names)
Preps_Nonstd

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54707,POP2|Chocolate Tart,40.0,piece,NaN,40.0,piece
1,P-22035,TP - DESSERT BAR APPLE CRUMBLE,1.0,slice,BAKING-RAW INGREDIENTS,1.0,slice
2,P-22028,TP - DESSERT BAR BUTTER TART,1.0,slice,BAKING-RAW INGREDIENTS,1.0,slice
3,P-22045,TP - DESSERT BAR LEMON CRUNCH,1.0,slice,BAKING-RAW INGREDIENTS,1.0,slice
4,P-22047,TP - DESSERT BAR MIXED BERRY,1.0,slice,BAKING-RAW INGREDIENTS,1.0,slice
5,P-32073,TP - DESSERT BUTTER TART,1.0,ea,BAKING-RAW INGREDIENTS,1.0,ea
6,P-22036,TP - DESSERT CAKE BLACK VELVET,1.0,slice,BAKING-RAW INGREDIENTS,1.0,slice
7,P-22031,TP - DESSERT CAKE BROWNIE BOMB,1.0,slice,BAKING-RAW INGREDIENTS,1.0,slice
8,P-22043,TP - DESSERT CAKE CHEESECAKE,1.0,slice,BAKING-RAW INGREDIENTS,1.0,slice
9,P-22046,TP - DESSERT CAKE MAPLE CHOCO,1.0,slice,BAKING-RAW INGREDIENTS,1.0,slice


In [55]:
path = os.path.join(os.getcwd(), "data", "cleaning", "Preps_NonstdUom.csv")
Preps_Nonstd.to_csv(path, index = False, header = True)

In [56]:
update_prep = pd.read_csv("data/cleaning/update/Preps_UpdateUom.csv")
update_prep

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54697,LEMON|Wedge 1/8,8.00,each,PREP,84.000,g
1,P-35132,MARINATED|Lemon & Herb Chx,185.00,ea,PREP,24050.000,g
2,P-51992,YIELD|Bread|Sourdough 5/8,36.00,slice,NaN,1620.000,g
3,P-26234,BATCH|Roasted Garlic Bread,16.00,ea,PREP,1280.000,g
4,P-26170,GRILLED|NaanBread,1.00,ea,PREP,125.000,g
5,P-16305,YIELD|Smokie (1pc),1.00,ea,NaN,112.000,g
6,P-26047,BOILED|Hard Boiled Eggs FT,50.00,ea,PREP,2500.000,g
7,P-26631,BRK|Pancake|Chocolate Chip,24.00,ea,NaN,4080.000,g
8,P-26057,FRIED|Sunny/Overeasy Egg,1.00,ea,ZDONT USE OK - PREP,50.000,g
9,P-26056,GRILLED|Tomato,1.00,ea,ZDONT USE OK - PREP,90.000,g


***

## New Items

In [57]:
# Load current Items List with assigned Emission Factors Category ID
Items_Assigned = pd.read_csv(os.path.join(os.getcwd(), "data", "mapping", "Items_List_Assigned.csv"))
Items_Assigned.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT


In [58]:
Items_Assigned.shape

(2077, 8)

### Get the List of New Items

In [59]:
# Filter new items by itemID that are not in the database and output them in a dataframe
col_names = list(Items.columns.values)
New_Items_List = []

for index, row in Items.iterrows():
    ItemId = Items.loc[index,'ItemId']
    if ItemId not in Items_Assigned['ItemId'].values:
        Dict = {}
        Dict.update(dict(row))
        New_Items_List.append(Dict)

New_Items = pd.DataFrame(New_Items_List, columns = col_names)

In [60]:
New_Items.insert(1, "CategoryID", '')
New_Items

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup


In [61]:
New_Items.shape

(0, 8)

In [62]:
# Store the list of new items into .csv file
# If New_Items is not empty then we convert it to a csv file. 
if not New_Items.empty:
    path = os.path.join(os.getcwd(), "data", "mapping", "new items", str(datetime.date(datetime.now()))+"_New_Items.csv")
    New_Items.to_csv(path, index = False, header = True)

In [63]:
file = pd.read_csv("data/mapping/new items/2022-11-01_New_Items.csv")
file.to_excel("2022-11-01_New_Items.xlsx",index = None, header=True)

In [64]:
file2 = pd.read_excel("data/mapping/new items added/New_Items_Added_11.xlsx")
file2.to_csv("data/mapping/new items added/New_Items_Added_11.csv", index=False)

***
## Data Summary

In [65]:
datasum = pd.DataFrame([New_Items.shape, Preps_Nonstd.shape, Items_Nonstd.shape],
                       columns = ['count', 'columns'], 
                       index = ['New_Items', 'Preps_Nonstd', 'Items_Nonstd'])
datasum

,count,columns
New_Items,0,8
Preps_Nonstd,24,7
Items_Nonstd,260,6


In [66]:
print(New_Items.columns)

Index(['ItemId', 'CategoryID', 'Description', 'CaseQty', 'CaseUOM', 'PakQty',
       'PakUOM', 'InventoryGroup'],
      dtype='object')
